In [1]:
import pandas as pd
import json
import time

In [2]:
file_path = "./output/list_of_products.xlsx"
df = pd.ExcelFile(file_path)
data = df.parse("Sheet1")

data.head()

,product_id,product_name,category_id,product_type_id,description,variant_id,variant_name,channel_id,channel_name,currency,price
0,UHJvZHVjdDo1MDI=,1121 Sella Basmati,Q2F0ZWdvcnk6OTA=,UHJvZHVjdFR5cGU6NzI=,• 1121 Sella Basmati Rice is known for its lon...,UHJvZHVjdFZhcmlhbnQ6Nzg2,1kg,Q2hhbm5lbDox,Default Channel,USD,37.00
1,UHJvZHVjdDo5MzA=,4MIX,Q2F0ZWdvcnk6OTI=,UHJvZHVjdFR5cGU6Mjc1,4Mix bar with crispy layers coated in white ch...,UHJvZHVjdFZhcmlhbnQ6MTEzMQ==,Default,Q2hhbm5lbDox,Default Channel,USD,9.99
2,UHJvZHVjdDo5NDY=,5 Cereals Biscuit,Q2F0ZWdvcnk6OTI=,UHJvZHVjdFR5cGU6Mjg1,"Florbiu biscuits with 5 cereals and chocolate,...",UHJvZHVjdFZhcmlhbnQ6MTE0OQ==,Default,Q2hhbm5lbDox,Default Channel,USD,46.00
3,UHJvZHVjdDo2Mjk=,7 Up,Q2F0ZWdvcnk6MTAw,UHJvZHVjdFR5cGU6MTM4,Beverage with a refreshing lemon-lime flavor.,UHJvZHVjdFZhcmlhbnQ6NzU5,175 ml,Q2hhbm5lbDox,Default Channel,USD,49.00
4,UHJvZHVjdDo2Mjk=,7 Up,Q2F0ZWdvcnk6MTAw,UHJvZHVjdFR5cGU6MTM4,Beverage with a refreshing lemon-lime flavor.,UHJvZHVjdFZhcmlhbnQ6NzYx,1 L,Q2hhbm5lbDox,Default Channel,USD,49.00


In [3]:
def format_description(text: str):
    return json.dumps({
        "time": int(time.time() * 1000),
        "blocks": [
            {
                "type": "paragraph",
                "data": {"text": text}
            }
        ],
        "version": "2.30.7"
    }, ensure_ascii=False)

In [4]:
products = []
for pid, group in data.groupby("product_id"):
    product_info = group.iloc[0]  # take first row for main product data
    
    product_dict = {
        "category_id": product_info["category_id"],
        "description": format_description(product_info["description"]),
        "id": product_info["product_id"],
        "name": product_info["product_name"],
        "product_type_id": product_info["product_type_id"],
        "variants": []
    }
    
    # Add all variants with channel listings
    for _, row in group.iterrows():
        variant = {
            "variant_id": row["variant_id"],
            "variant_name": row["variant_name"],
            "channelListings": [
                {
                    "channel_id": row["channel_id"],
                    "channel_name": row["channel_name"],
                    "currency": row["currency"],
                    "price": float(row["price"])
                }
            ]
        }
        product_dict["variants"].append(variant)
    
    products.append(product_dict)

In [5]:
output_path = "./input/products_export.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(products, f, ensure_ascii=False, indent=2)

print(f"✅ JSON file created: {output_path}")

✅ JSON file created: ./input/products_export.json
